<a href="https://colab.research.google.com/github/nobertomaciel/PLN-ANIMA/blob/main/PLN_Analise_de_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================================
# Exemplo prático de Processamento de Linguagem Natural (PLN)
# Aplicação: Análise de sentimentos de avaliações de produtos
# Técnicas: Tokenização, Stopwords, Bag-of-Words, TF-IDF, Word2Vec
# ===============================================

import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords

# Baixar recursos necessários do NLTK (apenas na primeira execução)
nltk.download('punkt')
nltk.download('stopwords')

# ===============================================
# 1️⃣ Exemplo de dataset realista
# ===============================================
avaliacoes = [
    "O celular é ótimo, tem uma câmera excelente!",
    "Horrível! Que produto terrível, não recomendo a ninguém.",
    "Entrega rápida e produto de boa qualidade.",
    "A bateria dura pouco e esquenta muito. Péssimo!",
    "Muito bom, atendeu todas as minhas expectativas.",
]

# ===============================================
# 2️⃣ Função de pré-processamento
# ===============================================
def preprocessar_texto(texto):
    texto = texto.lower()                               # minúsculas
    texto = re.sub(r'[^a-zA-Zà-úÀ-Ú\s]', '', texto)     # remove pontuação e números
    tokens = nltk.word_tokenize(texto, language='portuguese')  # tokenização
    stop_words = set(stopwords.words('portuguese'))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]  # remove stopwords
    return " ".join(tokens)

avaliacoes_limpo = [preprocessar_texto(a) for a in avaliacoes]

print("🔹 Textos após pré-processamento:")
print(avaliacoes_limpo)
print()

# ===============================================
# 3️⃣ Representação Bag-of-Words
# ===============================================
bow_vectorizer = CountVectorizer()
bow_matriz = bow_vectorizer.fit_transform(avaliacoes_limpo)

print("🔹 Representação Bag-of-Words (sparse matrix):")
print(pd.DataFrame(bow_matriz.toarray(), columns=bow_vectorizer.get_feature_names_out()))
print()

# ===============================================
# 4️⃣ Representação TF-IDF
# ===============================================
tfidf_vectorizer = TfidfVectorizer()
tfidf_matriz = tfidf_vectorizer.fit_transform(avaliacoes_limpo)

print("🔹 Representação TF-IDF:")
print(pd.DataFrame(tfidf_matriz.toarray(), columns=tfidf_vectorizer.get_feature_names_out()))
print()

# ===============================================
# 5️⃣ Representação Word2Vec
# ===============================================
# Para o Word2Vec, usamos listas de tokens (não texto único)
tokens_avaliacoes = [a.split() for a in avaliacoes_limpo]

modelo_w2v = Word2Vec(sentences=tokens_avaliacoes, vector_size=50, window=3, min_count=1, sg=1)
print("🔹 Vocabulário do modelo Word2Vec:")
print(list(modelo_w2v.wv.index_to_key))
print()

# Exemplo: similaridade entre palavras
print("🔹 Similaridade entre 'ótimo' e 'excelente':")
try:
    print(modelo_w2v.wv.similarity('ótimo', 'excelente'))
except KeyError:
    print("Alguma das palavras não foi encontrada no vocabulário.")

# ===============================================
# 6️⃣ Aplicação prática: vetorizar novas avaliações
# ===============================================
nova_avaliacao = "A qualidade da câmera é muito boa, mas a bateria dura pouco."
nova_avaliacao_proc = preprocessar_texto(nova_avaliacao)
nova_tfidf = tfidf_vectorizer.transform([nova_avaliacao_proc])

print("\n🔹 Nova avaliação pré-processada:")
print(nova_avaliacao_proc)
print("\n🔹 Vetor TF-IDF correspondente:")
print(pd.DataFrame(nova_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out()))
